In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import sys; sys.path.insert(0, '../')
import numpy as np

from invert import Solver
from invert.evaluate import true_variance_explained

from esinet.forward import create_forward_model, get_info
from esinet import Simulation

pp = dict(surface='white', hemi='both')

# Forward Model

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')
leadfield = fwd["sol"]["data"]
# leadfield, pos = unpack_fwd(fwd)[1:3]
# n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.3s remaining:    2.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


# Simulate

In [3]:
target_snr = 3
sim = Simulation(fwd, info, settings=dict(duration_of_trial=0.1, target_snr=target_snr, extents=25, number_of_sources=2)).simulate(2)
evoked = sim.eeg_data[0].average()
evoked.plot_joint()
stc = sim.source_data[0].copy()
stc.plot(**pp)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Lukas\Envs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
 50%|█████     | 1/2 [00:00<00:00,  2.03it/s]c:\Users\Lukas\Envs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
100%|██████████| 2/2 [00:00<00:00, 2005.88it/s]


source data shape:  (1284, 100) (1284, 100)


100%|██████████| 2/2 [00:00<00:00, 100.26it/s]

Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


Using pyvistaqt 3d backend.

Using control points [0.00000000e+00 6.16174809e-10 1.00246607e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Test

In [4]:
solver_name = "eloreta"

solver = Solver(solver_name, regularisation_method="L")
solver.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title="L-Curve"))
res_var = true_variance_explained(stc.data, stc_hat.data, leadfield)
print(f"Residual Variance: {res_var:.2f} %")

# solver = Solver(solver_name, regularisation_method="GCV")
# solver.make_inverse_operator(fwd, alpha="auto")
# stc_hat = solver.apply_inverse_operator(evoked)
# stc_hat.plot(**pp, brain_kwargs=dict(title="GCV"))
# res_var = true_variance_explained(stc.data, stc_hat.data, leadfield)
# print(f"Residual Variance: {res_var:.2f} %")

# solver = Solver(solver_name, regularisation_method="CRESO")
# solver.make_inverse_operator(fwd, alpha="auto")
# stc_hat = solver.apply_inverse_operator(evoked)
# stc_hat.plot(**pp, brain_kwargs=dict(title="CRESO"))
# res_var = true_variance_explained(stc.data, stc_hat.data, leadfield)
# print(f"Residual Variance: {res_var:.2f} %")

# solver = Solver(solver_name, regularisation_method="Product")
# solver.make_inverse_operator(fwd, alpha="auto")
# stc_hat = solver.apply_inverse_operator(evoked)
# stc_hat.plot(**pp, brain_kwargs=dict(title="Product"))
# res_var = true_variance_explained(stc.data, stc_hat.data, leadfield)
# print(f"Residual Variance: {res_var:.2f} %")

KeyboardInterrupt: 

In [60]:
import numpy as np
r_values = np.insert(np.logspace(-10, 10, 50), 0, 0)

M = evoked.data
n_chans, n_time  = M.shape

eigenvals = np.linalg.eig(leadfield @ leadfield.T)[0]
alphas = [r_value * np.max(eigenvals) / 2e4 for r_value in r_values]
inverse_operators = []
for alpha in alphas:
    inverse_operator = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans))
    inverse_operators.append(inverse_operator)

In [ ]:
gcv_values = []
for inverse_operator, alpha in zip(inverse_operators, alphas):
    x = inverse_operator@M
    nom = np.linalg.norm(leadfield@x - M)
    denom = np.trace(np.identity(n_chans) - leadfield @ inverse_operator)**2
    
    gcv_value = nom/denom
    gcv_values.append(gcv_value)


In [ ]:
import matplotlib.pyplot as plt
optimum_idx = np.argmin(gcv_values)
plt.figure()
plt.loglog(alphas, gcv_values)
plt.plot(alphas[optimum_idx], gcv_values[optimum_idx], '*')
plt.title(f"Alpha = {alphas[optimum_idx]:.1f}")

In [ ]:
y_hat = inverse_operators[optimum_idx]@M
stc_hat.data = y_hat
stc_hat.plot(**pp)
res_var = true_variance_explained(stc.data, stc_hat.data, leadfield)
print(f"Residual Variance: {res_var:.2f} %")